# Toy example of ImagePixelClassifier and ImagePatchClassifier

This notebook shows the usage of the image classsifiers on a toy example 

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import itertools

from eolearn.ml_tools import ImagePixelClassifier, ImagePatchClassifier

## Create dummy patch classifier

- predict: returns maximum value in an array
- predict_proba: returns maxiumum and minimum value in an array

In [2]:
class DummyPatchClassifier():
    def __init__(self, receptive_field):
        self.receptive_field = receptive_field
    
    def _predict(self, X):
        return np.max(X)
    
    def _predict_proba(self, X):
        return np.max(X), np.min(X)
    
    def predict(self, X):        
        if X.shape[1:3] == self.receptive_field:
            return np.asarray([self._predict(example) for example in X],dtype=np.int)
        else:
            raise ValueError('Dummy Classifier: input of incorrect shape')
        
    def predict_proba(self, X):
        if X.shape[1:3] == self.receptive_field:
            return np.asarray([self._predict_proba(example) for example in X],dtype=np.float)
        else:
            raise ValueError('Dummy Classifier: input of incorrect shape')

## Create dummy data set

50 images of size 10 x 10 x 3 bands

In [3]:
X = np.random.randint(0,100,(50,10,10,3))

In [4]:
X.shape

(50, 10, 10, 3)

In [5]:
X[0,:,:,2]

array([[ 4, 18, 15, 41, 76, 14, 72, 97, 37, 78],
       [72, 12, 48, 70, 35, 93, 51, 85,  7, 86],
       [73, 24, 88, 49, 66, 73, 22,  6, 28, 75],
       [38, 33, 97, 26, 67, 62, 93,  5,  7, 61],
       [76,  3,  3, 98, 32, 12, 36, 84, 50,  9],
       [89, 27, 63, 37, 85, 49, 63, 78, 65, 42],
       [76, 31, 81,  4,  7, 33, 38, 61, 70, 16],
       [38, 45, 42, 95, 31,  9, 55, 89, 89, 66],
       [86,  4,  4, 43, 78, 76, 88, 65, 73, 49],
       [19, 54, 40, 81, 49, 46, 65, 23, 53,  1]])

## Create `ImagePatchClassifier` and initialise it with `DummyPatchClassifier`

In [6]:
receptive_field = (2,2)
dummy_classifier = DummyPatchClassifier(receptive_field=receptive_field)

In [7]:
patch_classifier = ImagePatchClassifier(dummy_classifier, dummy_classifier.receptive_field)

### `predict` (run classifier over all patches and collect results)

In [8]:
Xprediction = patch_classifier.image_predict(X)

In [9]:
Xprediction.shape

(50, 10, 10)

In [10]:
Xprediction[0]

array([[86., 86., 90., 90., 93., 93., 97., 97., 96., 96.],
       [86., 86., 90., 90., 93., 93., 97., 97., 96., 96.],
       [85., 85., 97., 97., 97., 97., 99., 99., 81., 81.],
       [85., 85., 97., 97., 97., 97., 99., 99., 81., 81.],
       [89., 89., 98., 98., 85., 85., 84., 84., 83., 83.],
       [89., 89., 98., 98., 85., 85., 84., 84., 83., 83.],
       [98., 98., 99., 99., 97., 97., 90., 90., 89., 89.],
       [98., 98., 99., 99., 97., 97., 90., 90., 89., 89.],
       [97., 97., 88., 88., 84., 84., 95., 95., 91., 91.],
       [97., 97., 88., 88., 84., 84., 95., 95., 91., 91.]])

In [11]:
is_equal = True
for i, j, k in itertools.product(range(0,5), range(0,5), range(0,50)):
    if int(Xprediction[k,2*i,2*j]) != np.max(X[k,2*i:2*i+2,2*j:2*j+2,:]):
        print(Xprediction[k,i,j])
        print(np.max(X[k,2*i:2*i+2,2*j:2*j+2,:]))
        is_equal = False
        break
    
print(is_equal)

True


### `predict_proba` (run classifier over all patches and collect results)

In [12]:
Xprobabilities = patch_classifier.image_predict_proba(X)

In [13]:
Xprobabilities.shape

(50, 10, 10, 2)

In [14]:
np.array_equal(Xprediction,Xprobabilities[:,:,:,0])

True

In [15]:
is_equal = True
for i, j, k in itertools.product(range(0,5), range(0,5), range(0,50)):
    if int(Xprobabilities[k,2*i,2*j,1]) != np.min(X[k,2*i:2*i+2,2*j:2*j+2,:]):
        is_equal = False
        break
    
print(is_equal)

True


## Create `DummyPixelClassifier` and `ImagePixelClassifier`

In [16]:
class DummyPixelClassifier():
    def _predict(self, X):
        return np.max(X)
    
    def _predict_proba(self, X):
        return np.max(X), np.min(X)
    
    def predict(self, X):
        return np.asarray([self._predict(example) for example in X],dtype=np.int)
        
    def predict_proba(self, X):
        return np.asarray([self._predict_proba(example) for example in X],dtype=np.float)

In [17]:
dummy_pixel_classifier = DummyPixelClassifier()

In [18]:
pixel_classifier = ImagePixelClassifier(dummy_pixel_classifier)

### `predict` (run classifier over all patches and collect results)

In [19]:
Xprediction = pixel_classifier.image_predict(X)

In [20]:
Xprediction.shape

(50, 10, 10)

In [21]:
np.array_equal(Xprediction, np.max(X,axis=3))

True

### `predict_proba` (run classifier over all patches and collect results)

In [22]:
Xprobabilities = pixel_classifier.image_predict_proba(X)

In [23]:
Xprobabilities.shape

(50, 10, 10, 2)

In [24]:
np.array_equal(Xprobabilities[...,1], np.min(X,axis=3))

True